<a href="https://colab.research.google.com/github/BKirill/MLatFIAN2020_homework/blob/main/MLatFIAN2020_seminar04_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Task 1

(Titanic data again)

Build a model with `sklearn`'s `LogisticRegression` or `SVC` to get the accuracy of at least 0.81 on the test set. Can you get higher? 0.84?

Some (optional) suggestions:
- Add new features (e.g. missing value indicator columns)
- Fill missing values
- Encode categorical features (e.g. one-hot encoding)
- Scale the features (e.g. with standard or robust scaler)
- Think of other ways of preprocessing the features (e.g. `Fare` $\to$ `log(Fare)`)
- Try adding polynomial features
- use `sklearn.model_selection.GridSearchCV` to search for the best hyperparameters (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)



In [ ]:
!wget https://github.com/HSE-LAMBDA/MLDM-2020/raw/master/day-01/train.csv

--2020-10-12 14:54:14--  https://github.com/HSE-LAMBDA/MLDM-2020/raw/master/day-01/train.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/HSE-LAMBDA/MLDM-2020/master/day-01/train.csv [following]
--2020-10-12 14:54:14--  https://raw.githubusercontent.com/HSE-LAMBDA/MLDM-2020/master/day-01/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  58.89K  --.-KB/s    in 0.03s   

2020-10-12 14:54:15 (2.20 MB/s) - ‘train.csv’ saved [60302/60302]



In [2]:
import pandas as pd
data = pd.read_csv("train.csv", index_col='PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### About the data
Here's some of the columns
* Name - a string with person's full name
* Survived - 1 if a person survived the shipwreck, 0 otherwise.
* Pclass - passenger class. Pclass == 3 is cheap'n'cheerful, Pclass == 1 is for moneybags.
* Sex - a person's gender
* Age - age in years, if available
* SibSp - number of siblings on a ship
* Parch - number of parents on a ship
* Fare - ticket cost
* Embarked - port where the passenger embarked
 * C = Cherbourg; Q = Queenstown; S = Southampton

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def feature_selection_and_preprocessing(dataset):
    # Pre-processing the features
    features = dataset[["Fare", "Parch", "Sex", "SibSp", "Pclass", "Age", "Embarked", "Cabin"]].copy()
    features["AgeIsMissing"] = features.Age.isna().astype("int32")
    features["Age"] = features.Age.fillna(features.Age.median())
    features["Age"] = np.log(features["Age"])
    features["Fare"] = np.log(features["Fare"]+1)
    features["Embarked"] = data.Embarked.fillna("unknown")
    features["Cabin"] = features.Cabin.fillna("Z")
    features["Deck"] = features["Cabin"].map(lambda x: x[:1])
    features["AgeIsMissing"] = features.Age.isna().astype('int32')

    # Selecting the features
    features = features[["Parch", "SibSp", "Age", "Fare", "Embarked", 
                         "Pclass", "Sex", "AgeIsMissing", "Deck"]]
    return features

model = make_pipeline(
    make_column_transformer(
        (OneHotEncoder(sparse=False), ['Embarked']),
        (OneHotEncoder(sparse=False), ['Sex']),
        (OneHotEncoder(sparse=False), ['Deck']),
        remainder='passthrough'
    ),
    RobustScaler(),
    SVC(C=0.6)
)


# Validation code (do not touch)
data = pd.read_csv("train.csv", index_col='PassengerId')
data_train, data_test = train_test_split(data, test_size=200, random_state=42)

model.fit(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    ),
    data_train['Survived']
)

train_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    )
)

test_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_test.drop('Survived', axis=1)
    )
)

print("Train accuracy:", accuracy_score(
    data_train['Survived'],
    train_predictions
))
print("Test accuracy:", accuracy_score(
    data_test['Survived'],
    test_predictions
))

Train accuracy: 0.8350217076700435
Test accuracy: 0.825
